# 1. Routes

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp bmtc.apis.routes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import json
import time
import datetime
from tqdm import tqdm

import requests
import pandas as pd
from fastcore.all import Path
from traffic_data_bengaluru.utils import *
from nbdev.config import get_config

In [ ]:
#| export
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
#| hide
#| eval: false
data_directory = get_data_directory() / "bmtc"

# Functions

In [ ]:
#| export
def fetch_routes(pattern: str = "", sleep_duration: float = 0.1):
    """Fetches BMTC routes matching the given pattern, or all routes if pattern is empty."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/SearchRoute_v2"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    if pattern != '':
        payload = {"routetext": pattern}
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None
    else:
        routes = []
        characters = string.digits + string.ascii_lowercase
        for pattern in tqdm(characters, desc = 'Fetching routes'):
            routes += fetch_routes(pattern)
        return routes

In [ ]:
#| hide
#| eval: false

routes = fetch_routes(pattern = '210-NA')
print(json.dumps(routes, indent=4))

In [ ]:
#| export
def process_routes(routes):
    """Process and clean route data, returning a DataFrame with `route_id` and `route_number`."""
    df_routes = pd.DataFrame(routes)
    df_routes = df_routes.drop_duplicates(subset=["routeno"], keep="first")

    df_routes.rename(columns = {'routeno': 'route_number', 'routeparentid': 'route_id'}, inplace=True)
    df_routes = df_routes.sort_values(by='route_id').reset_index(drop=True)
    
    columns = ['route_id', 'route_number']
    return df_routes[columns]

In [ ]:
#| hide
#| eval: false
df_routes = process_routes(routes)
df_routes

In [ ]:
#| export
def task_fetch_routes(data_directory):
    logger.info("Fetching routes ...")
    
    filename = f'{str(int(datetime.datetime.now().timestamp()))}'    
    routes = fetch_routes()

    filepath = data_directory / 'raw' / 'routes' / f'{filename}.json'
    filepath.parent.mkdir(parents=True, exist_ok=True)
    with open(filepath, 'w') as f:
        json.dump(routes, f, indent=2)
    logger.info(f"Raw routes saved successfully to {filepath}")

    logger.info("Processing routes ...")
    df_routes = process_routes(routes)

    filepath = data_directory / 'cleaned' / 'routes' / f'{filename}.csv'
    filepath.parent.mkdir(parents=True, exist_ok=True)
    df_routes.to_csv(filepath, index=False)
    logger.info(f"Processed routes saved successfully to {filepath}")

In [ ]:
#| hide
#| eval: false

task_fetch_routes(data_directory=data_directory)

In [ ]:
#| export
def get_routes(data_directory: Path):
    filepath = extract_file(get_latest_file(data_directory / "cleaned" / "routes"))
    routes = pd.read_csv(filepath).drop_duplicates(subset=["route_id"])
    return routes

In [ ]:
#| hide
#| eval: false
df_routes = get_routes(data_directory)

print(df_routes.shape)
df_routes

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()